In [ ]:
import pandas as pd
import numpy as np
import jupyter_black
from typing import Literal, Tuple
from IPython.display import HTML


import nfl_data_py as nfl
from gspread.exceptions import WorksheetNotFound, SpreadsheetNotFound

from src.modelling.metrics import success_rate_lambda
from src.modelling.utils import guess_the_lines_ovr, get_week_spreads
from src.modelling.homers import (
    google_sheet_names,
    pickers,
    master_google_sheet,
    calc_percentile_to_gpf,
    orchestrate_power_ratings_to_picks,
    get_power_ratings,
)
from src.scraping.google_sheets import col_to_int
from src.utils.connections import load_service_account
from src.utils.teams import standardize_teams
from src.utils.logos import get_team_logo, get_logo_url
from src.visualisation.plots import plot_team_scatter
from src.visualisation.tables import pg_cmap

jupyter_black.load()
pd.set_option("display.max_columns", None)

In [ ]:
week = 10
picker = "Griffin"
# picker = "Harry"
# picker = "Chuck"

In [ ]:
gtl = orchestrate_power_ratings_to_picks(week=week, picker=picker, overwrite_tab=False)

In [ ]:
pickers = ["Griffin", "Harry", "Chuck"]
# pickers = ["Griffin"]

In [ ]:
power_dfs = [
    get_power_ratings(week=week, picker=picker)["net_perc"].to_frame().sort_index()
    for picker in pickers
]
composite = power_dfs[0].copy()
for df in power_dfs[1:]:
    composite += df
composite = composite / len(power_dfs)
power_dfs.insert(0, composite)
composite_df = pd.concat(power_dfs, axis=1)
composite_df.columns = ["Composite"] + pickers
composite_df = composite_df.sort_values("Composite", ascending=False)

composite_rank = composite_df.rank(ascending=False).astype(int)
composite_rank = composite_rank.reset_index()
composite_rank["team"] = composite_rank["team"].apply(get_logo_url)
composite_rank = composite_rank.set_index("team")
# composite_rank.index.name = ("Team", "")
# Display the DataFrame as HTML in Jupyter Notebook
composite_rank = composite_rank.style.background_gradient(cmap=pg_cmap)
html = composite_rank.to_html(escape=False)  # Escape=False ensures HTML rendering
display(HTML(html))

In [ ]:
composite_df = power_dfs[0]
composite_df["net_gpf"] = composite_df["net_perc"].apply(calc_percentile_to_gpf)
composite_df

In [ ]:
guess_the_lines_ovr(composite_df, 4).sort_values("rank")